In [1]:
#|default_exp concurrent.remote.test_RemoteController_0

In [2]:
#|export
import asyncio, os, sys, subprocess
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, find_available_port
from fbdev.dev_utils import is_in_repl
from fbdev.concurrent.remote import RemoteController, get_client

In [3]:
#|export
port_num = find_available_port()

remote_py_script = """
address = ('localhost', %s)
authkey = b'secret'

import asyncio, os, multiprocessing
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager
from fbdev.concurrent.remote import RemoteController

listener = Listener(address, authkey=authkey)
async def main(conn):
    task_manager = TaskManager(None)
    child_values = {}
    def send_val_to_child(val): child_values['val'] = val
    remote = RemoteController(conn, task_manager,
        main={
            'msg_from_parent' : lambda msg: print(f"Child received: '{msg}'"),
            'send_val_to_child' : send_val_to_child,
        }
    )
    
    remote.send_ready()
    await remote.await_ready()
    
    await task_manager.exec_coros(
        remote.do('main', 'msg_from_child', 'Hello from child!'),
        asyncio.sleep(0.1),
        remote.do('main', 'send_val_to_parent', 'from_child'),
        asyncio.sleep(0.5),
        sequentially=True
    )
    assert child_values['val'] == 'from_parent'

with listener.accept() as conn:
    asyncio.run(main(conn))
""" % port_num

address = ('localhost', port_num)
authkey = b'secret'

async def test():
    task_manager = TaskManager(None)
    proc = subprocess.Popen([sys.executable, '-c', remote_py_script])

    with get_client(address, authkey=authkey) as conn:
        parent_values = {}
        def send_val_to_parent(val): parent_values['val'] = val
        remote = RemoteController(conn, task_manager,
            main={
                'msg_from_child' : lambda msg: print(f"Parent received: '{msg}'"),
                'send_val_to_parent' : send_val_to_parent,
            }
        )
        
        remote.send_ready()
        await task_manager.exec_coros(
            remote.await_ready(),
            asyncio.sleep(0.1),
            remote.do('main', 'msg_from_parent', 'Hello from parent!'),
            asyncio.sleep(0.1),
            remote.do('main', 'send_val_to_child', 'from_parent'),
            asyncio.sleep(0.1),
            sequentially=True
        )

        assert parent_values['val'] == 'from_child'
        
    await task_manager.destroy()
    proc.communicate()  # This will wait until the process completes

In [4]:
await test()

Parent received: 'Hello from child!'Child received: 'Hello from parent!'

